In [1]:
import glob
import rasterio
from pathlib import Path
import os
import reverse_geocoder as rg
import pyproj
import geopandas as gpd
import sys

In [2]:
if Path.cwd().name != 'src':
    os.chdir('./src')
from maxarseg import build

In [3]:
from maxarseg import build
events_names = build.get_all_events()
for i, e_n in enumerate(events_names):
    print(i, e_n)

0 southafrica-flooding22
1 Kalehe-DRC-Flooding-5-8-23
2 shovi-georgia-landslide-8Aug23
3 Emilia-Romagna-Italy-flooding-may23
4 src
5 NWT-Canada-Aug-23
6 Libya-Floods-Sept-2023
7 Gambia-flooding-8-11-2022
8 New-Zealand-Flooding23
9 Hurricane-Fiona-9-19-2022
10 Hurricane-Idalia-Florida-Aug23
11 Hurricane-Ian-9-26-2022
12 yellowstone-flooding22
13 pakistan-flooding22
14 Morocco-Earthquake-Sept-2023
15 Indonesia-Earthquake22
16 cyclone-emnati22
17 Kahramanmaras-turkey-earthquake-23
18 test_folder
19 Marshall-Fire-21-Update
20 metadata
21 India-Floods-Oct-2023
22 Maui-Hawaii-fires-Aug-23
23 tonga-volcano21
24 BayofBengal-Cyclone-Mocha-May-23
25 Sudan-flooding-8-22-2022


In [4]:
bs = 2
config = build.SegmentConfig(batch_size = bs, device='cuda')

/home/vaschetti/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased
- GD model device: cuda:0
- Efficient SAM device: cuda:0


In [5]:
evento = build.Event(events_names[7], seg_config = config, when='pre')
print(evento.region_name)
all_mosaics_names = evento.all_mosaics_names
print(all_mosaics_names)
m0 = evento.mosaics[all_mosaics_names[0]]
print(f'Number of tiles: {m0.tiles_num}')

AfricaWest-Full
['10300100CFC9A500', '105001002BD68F00', '1040010073D77D00']
Number of tiles: 18


In [6]:
m0.set_build_gdf()

Found 3 links matching: [33303200, 333032, 3330320]


In [7]:
m0.proj_road_gdf

In [8]:
m0.tiles_paths[0]

PosixPath('/mnt/data2/vaschetti_data/maxar/Gambia-flooding-8-11-2022/pre/10300100CFC9A500/033133031231.tif')

In [9]:
m0.set_road_gdf()

not USA:  AfricaWest-Full.tsv


In [14]:
road_mask = m0.get_tile_road_mask_np(m0.tiles_paths[0])

array([0, 1], dtype=uint8)

In [16]:
import rasterio
from rasterio.windows import Window
from maxarseg import geoDatasets
from maxarseg import samplers
from maxarseg import segment
from torch.utils.data import DataLoader
from torchgeo.datasets import stack_samples, unbind_samples
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from maxarseg import samplers_utils
import torch
from time import time

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [17]:
def segment_tile(self, tile_path):
    seg_config = self.event.seg_config

    dataset = geoDatasets.Maxar(str(tile_path))
    sampler = samplers.MyBatchGridGeoSampler(dataset, batch_size=seg_config.batch_size, size=seg_config.size, stride=seg_config.stride)
    dataloader = DataLoader(dataset , batch_sampler=sampler, collate_fn=stack_samples)

    canvas = np.zeros((seg_config.size, seg_config.size, 3), dtype=np.uint8)

    i = 0
    start_time_all = time()
    for batch in tqdm(dataloader):
        if i == 5:
            break
        if i != 4:
            i+=1
            continue
        original_img_tsr = batch['image']
        img_b = batch['image'].permute(0,2,3,1).numpy().astype('uint8') #TODO: l'immagine viene convertita in numpy ma magari è meglio lasciarla in tensor

        #trees
        GD_t_0 = time()
        tree_boxes_b, num_trees4img = segment.get_GD_boxes(img_b, seg_config.GD_model,
                                            seg_config.TEXT_PROMPT,
                                            seg_config.BOX_TRESHOLD,
                                            seg_config.TEXT_TRESHOLD,
                                            dataset.res,
                                            device = seg_config.device,
                                            max_area_mt2 = seg_config.max_area_GD_boxes_mt2)
        print('GD_time: ', time() - GD_t_0)
        print('- GD model device:', next(seg_config.GD_model.parameters()).device)
        
        #building            
        print(batch['bbox'])            
        building_boxes_b, num_build4img = segment.get_batch_buildings_boxes(batch['bbox'],
                                                                    proj_buildings_gdf = self.proj_build_gdf,
                                                                    dataset_res = dataset.res,
                                                                    ext_mt = 10)
        max_detect = max(num_trees4img + num_build4img)
        print(f'max_detect: {max_detect}')
        input_points, input_labels = build.get_input_pts_and_lbs(tree_boxes_b, building_boxes_b, max_detect)
        
        #boxes4FSam = build.get_boxes4FSam(tree_boxes_b, building_boxes_b, max_detect)
        #print('tree_b shape: ', building_boxes_b[0].shape)
        #print('input_points shape: ', input_points.shape)
        #print('input_labels shape: ', input_labels.shape)

        all_masks_b = segment.ESAM_from_inputs(original_img_tsr,
                                                torch.from_numpy(input_points),
                                                torch.from_numpy(input_labels),
                                                efficient_sam = seg_config.efficient_sam,
                                                device = seg_config.device,
                                                num_parall_queries = 17)
        
        tree_mask_b, building_mask_b, pad_mask_b = segment.discern(all_masks_b, num_trees4img, num_build4img)

        #roads
        """get_road_masks_b(batch['bbox'],
                         self.proj_road_gdf,
                         seg_config.size,
                         dataset.res,
                         ext_mt = 10)"""
        
        #road_mask_b = segment.get_road_mask(img_b,)
        i += 1
    print(f'\nTotal Time for {seg_config.batch_size * i} images: ', time() - start_time_all)
    return img_b, original_img_tsr#, tree_boxes_b, building_boxes_b, all_masks_b, tree_mask_b, building_mask_b, pad_mask_b

_, _ = segment_tile(m0, m0.tiles_paths[17])
#img_b, original_img, tree_boxes_b, building_boxes_b, all_masks_b, tree_mask_b, building_mask_b, pad_mask_b = segment_tile(m0, m0.tiles_paths[17])

  0%|          | 0/1682 [00:00<?, ?it/s]

  0%|          | 2/1682 [00:00<01:35, 17.58it/s]

In sampler
tile_polygon:  POLYGON ((315427.24609375 1485156.25, 320156.25 1485156.25, 320156.25 1479843.75, 315394.8974609375 1479843.75, 315427.24609375 1485156.25))


  0%|          | 4/1682 [00:04<33:06,  1.18s/it]

GD_time:  4.442540645599365
- GD model device: cuda:0
[BoundingBox(minx=316033.935546875, maxx=316217.041015625, miny=1479843.75, maxy=1480026.85546875, mint=0.0, maxt=9.223372036854776e+18), BoundingBox(minx=316125.48828125, maxx=316308.59375, miny=1479843.75, maxy=1480026.85546875, mint=0.0, maxt=9.223372036854776e+18)]
max_detect: 83
-1


ValueError: index can't contain negative values

In [ ]:
m0.segment_tile()